In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
from shapely.geometry import Polygon
from shapely import wkt

sys.path.append('../')
from scripts.match_tlid_geo import import_data
from scripts.match_tlid import match_county_tlid

# Plot figures inline
plt.ion()

In [ ]:
# Import Denver addresses and TIGER files
geo_points, geo_edges = import_data(county_code='08031')
geo_edges.TLID = geo_edges['TLID'].apply(str)
tlid_xwalk = pd.read_csv("../results/address_tlid_xwalk/08031_tlid_match.csv",
                         converters={'TLID_match': lambda x: str(x)})

tlid_geo_xwalk = pd.read_csv("../results/address_tlid_xwalk/08031_geo_match.csv",
                             converters={'TLID_match': lambda x: str(x)})
crs = {'init': 'epsg:4269'}

# Convert addresses to spatial object
tlid_geo_xwalk['geometry'] = tlid_geo_xwalk['geometry'].apply(wkt.loads)
addresses_geo_matched = gpd.GeoDataFrame(
    tlid_geo_xwalk, crs=crs, geometry=tlid_geo_xwalk['geometry'])

In [ ]:
# Merge address data with the xwalk
addresses_matched = pd.merge(geo_points, tlid_xwalk,  how='left', left_on=[
                             'MAFID'], right_on=['MAFID'])
print("\n\nNon-geo results:")
print(addresses_matched.head())
print("\n\nGeo results:")
print(addresses_geo_matched.head())
print("\n\nEdges:")
print(geo_edges.head())

In [ ]:
def validation_map(addresses, edges, tlid, fg):
    """
    Creates a folium map layer of address points and linear features in the edges file.
    Clips both datasets to the zoom boundary to allow map to more easily fit in memory.
    Parameters
    ----------
    addresses: gpd DataFrame
                contains point data of addresses
    edges: gpd DataFrame
            contains linear data, simplified or as downloaded from TIGER
    tlid: str
                TLID to group by -- will only plot points and lines with this TLID
    fg: folium map group
        named fg object to use to create layers

    Returns
    -------
    fg: folium map group
        map group of address points and edges lines, which can be added to a basemap
    """

    # Extract relevant address points and the correct line
    points_with_TLID = addresses.loc[addresses['TLID_match'] == tlid]
    edge_with_TLID = edges.loc[edges['TLID'] == tlid]

    # Interactive plots with Folium
    edges_json = edge_with_TLID.to_crs(epsg='4326').to_json()
    lines = folium.features.GeoJson(edges_json)
    fg.add_child(lines)

    addresses_json = points_with_TLID.to_crs(epsg='4326').to_json()
    points = folium.features.GeoJson(addresses_json)
    fg.add_child(points)

    return fg

## Map of points and lines organized by TLID

In [ ]:
random_tlids = addresses_matched['TLID_match'].sample(n=30).dropna().tolist()

# Import block data to check how well spatial join worked
blocks = gpd.read_file(
    "../data/tiger_csv/den_tiger/08031_faces/tl_2017_08031_faces.shp")

# Create base map
results_map = folium.Map([39.785, -104.83],
                         zoom_start=14,
                         tiles='cartodbpositron')


# Loop through random TLIDs and create a map layer for the results of each
for tlid in random_tlids:
    fg = folium.FeatureGroup(name=tlid)
    results_map.add_child(validation_map(
        addresses_matched, geo_edges, tlid, fg))

results_map.add_child(folium.LayerControl())
results_map

In [ ]:
# Create base map
geo_results_map = folium.Map([39.785, -104.83],
                             zoom_start=14,
                             tiles='cartodbpositron')


# Loop through random TLIDs and create a map layer for the results of each
for tlid in random_tlids:
    fg = folium.FeatureGroup(name=tlid)
    geo_results_map.add_child(validation_map(
        addresses_geo_matched, geo_edges, tlid, fg))

geo_results_map.add_child(folium.LayerControl())
geo_results_map